In [18]:
import json
import pandas as pd
import numpy as np
import re

In [19]:
df = pd.read_csv('data/schema2/deepseek_baseline.csv')
df

,Unnamed: 0.1,Unnamed: 0,id,full_text,filtered_text,json_output
0,0,0,115,Paper #: 115\r\nLiquid medium annealing for fa...,Liquid medium annealing for fabricating durabl...,"{'perovskite_composition': 'FA0.9Cs0.1PbI3', '..."
1,1,1,112,Paper #: 112\r\nIodine reduction for reproduci...,Iodine reduction for reproducible and high-per...,{'perovskite_composition': 'MA 0.7 FA 0.3 PbI3...
2,2,2,72,Paper #: 72\r\nSuppressed Ion Migration in Red...,T oday's perovskite solar cells (PSCs) require...,{'perovskite_composition': 'Cs 0.05 FA 0.8 MA ...
3,3,3,75,Paper #: 75\r\nDynamical Evolution of the 2D/3...,UV-VIS Absorption The absorption spectra of t...,"{'perovskite_composition': None, 'electron_tra..."
4,4,4,40,Paper #: 40\r\nDamp heat-stable perovskite sol...,Damp heat-stable perovskite solar cells with t...,"{'perovskite_composition': None, 'electron_tra..."
...,...,...,...,...,...,...
144,144,144,92,Paper #: 92\r\n\r\nBanners \r\n Banners New fo...,Electronic Table of Contents (eToC) \r\n Stan...,{'perovskite_composition': 'FA-based mixed per...
145,145,145,123,Paper #: 123\r\nConstructing heterojunctions b...,P erovskite solar cells (PSCs) have reached po...,"{'perovskite_composition': 'FAPbI3:0.95, MAPbB..."
146,146,146,31,Paper #: 31\r\n\r\nWe performed 1 H-NMR to sup...,We performed 1 H-NMR to support our proposed r...,"{'perovskite_composition': None, 'electron_tra..."
147,147,147,36,Paper #: 36\r\nUnderstanding Performance Limit...,Understanding Performance Limiting Interfacial...,NaN


In [20]:
def escape_internal_quotes(json_string):
    # This regex finds values within quotes and fixes internal unescaped quotes
    return re.sub(r'":\s*"([^"]*?)"', lambda m: '": "' + m.group(1).replace('"', '\\"') + '"', json_string)

In [21]:
def force_fix(json_string):
    '''
    {"perovskite_composition": "Cs0.05(FA0.98MA0.02)0.95Pb(I0.98Br0.02)3", "electron_transport_layer": "6,6"-phenyl-C61-butyric acid methyl ester", "pin_nip_structure": "NIP", "hole_transport_layer": "2-(3,6-dimethoxy-9H-carbazol-9-yl)ethyl phosphonic acid", "test_1": {"test_name": "ISOS-L", "temperature": "25", "time": "1000", "humidity": "50-60", "passivating_molecule": "β-poly(1,1-difluoroethylene)", "control_pce": "22.3", "treated_pce": "24.6", "control_voc": "1.13", "treated_voc": "1.18"}}
    '''
    ## convert {" pattern to ZS
    json_conv = re.sub(r'{\"',"ZS", json_string)
    ## convert ": to ZT
    json_conv = re.sub(r'\":', "ZT", json_conv)
    ## Convert _" to ZP
    json_conv = re.sub(r' \"', "ZP", json_conv)
    ## Convert ", to ZV
    json_conv = re.sub(r'\",Z', "ZVZ", json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub(r'\"}', "ZQ", json_conv)

    ##Perform real conversion interested
    json_conv = json_conv.replace('"', "'")

    #Revert Everything 

    ## convert ZS to {" 
    json_conv = re.sub("ZS", r'{"', json_conv)
    ## convert ": to ZT
    json_conv = re.sub("ZT", r'":', json_conv)
    ## Convert _" to ZP
    json_conv = re.sub("ZP", r' "', json_conv)
    ## Convert ", to ZV
    json_conv = re.sub("ZV", r'",', json_conv)
    ## Convert "} to ZQ
    json_conv = re.sub("ZQ", r'"}', json_conv)

    return json_conv

In [32]:
def clean_json(json_string):
    if json_string is None: return None
    
    if "```json" in json_string:
        json_string = json_string.split("```json")[1]
        json_match = re.search(r"\{.*\}", json_string, re.DOTALL)
        if json_match:
            json_string = json_match.group(0).strip()
    json_string = json_string.replace("None", "null")
    json_string = json_string.replace("'", "\"")
    json_string = escape_internal_quotes(json_string)
    json_string = force_fix(json_string)
    json_string = re.sub(r'(".*?")', lambda m: m.group(1).replace(":", ""), json_string)
    json_string = json_string.replace("treated_pec", "treated_pce")
    return json_string

In [34]:
import re
output = {}
to_format = {}
i = 0
for index, row in df.iterrows():
    i += 1
    json_output = row["json_output"]
    if json_output is None or json_output is np.nan:
        output[str(row["id"])] = None
        continue
    json_match = re.search(r"\{.*\}", json_output, re.DOTALL)
    if json_match:
        raw_json = json_match.group(0).strip()
        try:
            raw_json = json.loads(clean_json(raw_json))
        except json.JSONDecodeError as e:
            raw_json = None
            to_format[row["id"]] =clean_json(raw_json)
    else:
        raw_json = None

        
    output[str(row["id"])] = raw_json

In [36]:
with open('data/deepseek_baseline.json', 'w') as f:
    json.dump(output, f)

In [24]:
with open('data/final_extraction/to_format_formatted.json', 'r') as f:
    formatted = json.load(f)
    

In [26]:
output.update(formatted)

In [33]:
output = {k: v for k, v in output.items() if "test" not in k}

In [35]:
with open('data/final_extraction/final_extraction_formatted.json', 'w') as f:
    json.dump(output, f)

In [46]:
with open('data/final_extraction/final_extraction_formatted.json', 'r') as f:
    data = json.load(f)

In [60]:
to_format = {}
for key in data:
    if type(data[key]) == dict:
        continue
    try:
        json.loads(clean_json(data[key]))
        print(key)
    except json.JSONDecodeError as e:
        to_format[key] = clean_json(data[key])

In [62]:
with open('data/final_extraction/to_format.json', 'w') as f:
    json.dump(to_format, f)

In [64]:
import pandas as pd
df = pd.read_json('data/final_extraction/final_extraction_formatted.json')
data = df.T.sort_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2538 entries, 0 to 2538
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   perovskite_composition    1036 non-null   object
 1   electron_transport_layer  958 non-null    object
 2   pin_nip_structure         772 non-null    object
 3   hole_transport_layer      873 non-null    object
 4   passivating_molecule      2051 non-null   object
 5   control_pce               1229 non-null   object
 6   control_voc               996 non-null    object
 7   treated_pce               1592 non-null   object
 8   treated_voc               1159 non-null   object
 9   test_1                    2160 non-null   object
 10  test_2                    636 non-null    object
 11  structure_pin_nip         55 non-null     object
 12  treated_jsc               68 non-null     object
 13  treated_ff                93 non-null     object
 14  test_3                    210

In [65]:
data[(data["passivating_molecule"].isna() == False) & (data["control_pce"].isna() == False) & (data["treated_pce"].isna() == False)]

,perovskite_composition,electron_transport_layer,pin_nip_structure,hole_transport_layer,passivating_molecule,control_pce,control_voc,treated_pce,treated_voc,test_1,...,treated_eqe,treated_stabilized_pce,control_fill_factor,treated_fill_factor,controljsc,treatedjsc,treated_hysteresis_index,binding_energy,fill_factor,treated_iscc
0,MAPbI3,PEDOT:PSS,NIP,PCBM,DAAB,4.5,0.87,12.6,0.92,"{'test_name': 'ISOS-L', 'temperature': 30, 'ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Cs 0.05 FA 0.9 MA 0.05 PbI 3,None,None,None,Cyanoguanidine diiodide,20.44,1077,23.04,1119,"{'test_name': 'ISOS-D', 'temperature': None, '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cs 0.17 FA 0.83 Pb(I 0.9 Br 0.1 ) 3 (WBg),TiO2,NaN,Spiro-OMeTAD,tBBAI,14.91,1.07,23.5,1.14,"{'stability_type': 'ISOS-L-1', 'temperature': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,FAPbI3 0.85 (-MAPbBr3) 0.15,Spiro-OMeTAD,NIP,None,7-amino-4-trifluoromethylcoumarin (C151),20.44,1.09,22.46,1.14,"{'test_name': 'aging', 'temperature': 20, 'tim...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,None,None,None,None,ICBA,22.4,1.15,25.1,1.19,"{'test_name': None, 'temperature': 40, 'time':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,CsPbI3,TiO2,n-i-p,spiro-OMeTAD,n-octylammonium iodide (n-OAI),18.6,1.18,19.8,1.23,"{'test_name': 'ISOS-L', 'temperature': 25, 'ti...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2530,None,None,None,None,TDMASn,15,None,18.2,None,"{'test_name': 'Stability test', 'temperature':...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2532,MAPbI3,None,None,None,OTTM,18.45,1.08,21.01,1.12,"{'test_name': 'stability', 'temperature': 85, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2533,MAPbI3,PCBM,PIN,NiOx,PbS,15.26,1.002,18.74,1.025,"{'test_name': 'ISOS-T', 'temperature': 400, 't...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
has_passivator = data[data["passivating_molecule"].str.contains("perovskite_composition") == False]

In [74]:
has_passivator.to_csv('data/final_extraction/df_has_passivator.csv')